# Redshift Serverless에 가상 데이터 삽입

이 노트북은 가상의 고객 데이터를 생성하고 Redshift Serverless에 삽입하는 과정을 보여줍니다.

In [ ]:
import boto3
import pandas as pd
import numpy as np
from faker import Faker
from sqlalchemy import create_engine
from botocore.exceptions import ClientError

In [ ]:
# Redshift Serverless 연결 정보
redshift_serverless_namespace = "stoa-aiml-pipeline-namespace"
redshift_serverless_workgroup = "stoa-aiml-pipeline-workgroup"
database_name = "dev"  # 기본 데이터베이스 이름, 필요시 수정
db_user = "admin"  # 기본 사용자 이름, 필요시 수정
region = "ap-northeast-2"  # 서울 리전

In [ ]:
# Redshift Serverless 엔드포인트 가져오기
redshift_client = boto3.client('redshift-serverless', region_name=region)

try:
    response = redshift_client.get_workgroup(workgroupName=redshift_serverless_workgroup)
    endpoint = response['workgroup']['endpoint']['address']
    port = response['workgroup']['endpoint']['port']
    print(f"Redshift Serverless 엔드포인트: {endpoint}:{port}")
except ClientError as e:
    print(f"엔드포인트를 가져오는 데 실패했습니다: {e}")

In [ ]:
# 임시 자격 증명 가져오기
sts_client = boto3.client('sts')
response = sts_client.get_session_token()
credentials = response['Credentials']

# Redshift 연결 문자열 생성
conn_string = f"postgresql://{db_user}:{credentials['SessionToken']}@{endpoint}:{port}/{database_name}"

# SQLAlchemy 엔진 생성
engine = create_engine(conn_string)

In [ ]:
import numpy as np
import pandas as pd
from faker import Faker

fake = Faker()

def generate_customer_data(n):
    # 기본 데이터 생성
    data = {
        'customer_id': range(1, n+1),
        'name': [fake.name() for _ in range(n)],
        'email': [fake.email() for _ in range(n)],
        'age': np.random.normal(40, 15, n).astype(int),  # 평균 40세, 표준편차 15
        'city': np.random.choice(['Seoul', 'Busan', 'Incheon', 'Daegu', 'Daejeon'], n),
    }
    
    # 나이에 따른 구매 금액 경향성 추가
    base_purchase = np.random.normal(500, 200, n)
    age_factor = (data['age'] - 20) / 60  # 20세 ~ 80세 범위로 정규화
    data['purchase_amount'] = (base_purchase * (1 + age_factor)).round(2)
    data['purchase_amount'] = np.clip(data['purchase_amount'], 10, 2000)  # 10 ~ 2000 사이로 제한
    
    # 도시별 구매 패턴 추가
    city_factors = {'Seoul': 1.2, 'Busan': 1.0, 'Incheon': 0.9, 'Daegu': 0.95, 'Daejeon': 0.85}
    data['purchase_amount'] *= [city_factors[city] for city in data['city']]
    
    # 충성 고객 그룹 추가 (약 10%의 고객)
    loyal_customers = np.random.choice([0, 1], n, p=[0.9, 0.1])
    data['purchase_amount'] += loyal_customers * np.random.normal(300, 100, n)
    
    # 계절성 효과 추가 (예: 분기별 구매 패턴)
    seasons = np.random.choice(['Q1', 'Q2', 'Q3', 'Q4'], n)
    season_factors = {'Q1': 0.8, 'Q2': 1.0, 'Q3': 1.1, 'Q4': 1.3}
    data['season'] = seasons
    data['purchase_amount'] *= [season_factors[season] for season in seasons]
    
    # 최종 구매 금액 반올림
    data['purchase_amount'] = data['purchase_amount'].round(2)
    
    return pd.DataFrame(data)

# 10,000개의 가상 고객 데이터 생성
df = generate_customer_data(10000)
print(df.head())

# 데이터 요약 통계
print(df.describe())

# 도시별 평균 구매 금액
print(df.groupby('city')['purchase_amount'].mean())

# 나이와 구매 금액의 상관관계
print(f"나이와 구매 금액의 상관계수: {df['age'].corr(df['purchase_amount']):.2f}")

# 계절별 평균 구매 금액
print(df.groupby('season')['purchase_amount'].mean())

In [ ]:
# Redshift에 테이블 생성
create_table_query = """
CREATE TABLE IF NOT EXISTS customers (
    customer_id INT PRIMARY KEY,
    name VARCHAR(100),
    email VARCHAR(100),
    age INT,
    city VARCHAR(100),
    purchase_amount DECIMAL(10, 2)
);
"""

with engine.connect() as conn:
    conn.execute(create_table_query)
    print("테이블이 성공적으로 생성되었습니다.")

In [ ]:
# 데이터를 Redshift에 삽입
df.to_sql('customers', engine, if_exists='append', index=False, method='multi', chunksize=1000)
print("데이터가 성공적으로 삽입되었습니다.")

In [ ]:
# 데이터 확인
query = "SELECT * FROM customers LIMIT 5;"
result = pd.read_sql(query, engine)
result

In [ ]:
# 연결 종료
engine.dispose()
print("연결이 종료되었습니다.")